In [7]:
import numpy as np
import cv2
import time
import imutils


def draw_flow(img, flow, step=16):

    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T

    lines = np.vstack([x, y, x-fx, y-fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(img_bgr, lines, 0, (0, 255, 0))

    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(img_bgr, (x1, y1), 1, (0, 255, 0), -1)

    return img_bgr


def draw_hsv(flow):

    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]

    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)

    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return bgr

video_path = '.\\videos\\video2.mov'
cap = cv2.VideoCapture(video_path)

suc, prev = cap.read()
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
valor = 800
prevgray = imutils.resize(prevgray, width=min(valor, prevgray.shape[1]))

while True:
    suc, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = imutils.resize(gray, width=min(valor, gray.shape[1]))
    # start time to calculat FPS
    start = time.time()
    flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    prevgray = gray
    print(flow)
    # End time
    end = time.time()
    # calculate the FPS for current frame detection
    fps = 1 / (end-start)
    print(f"{fps:.2f} FPS")
    #cv2.imshow('flow', draw_flow(gray, flow))
    cv2.imshow('flow HSV', draw_hsv(flow))
    key = cv2.waitKey(5)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

[[[ 6.3196593e-04 -3.2025997e-03]
  [-2.4229765e-03 -5.3320592e-03]
  [-1.0115662e-02 -9.5888339e-03]
  ...
  [ 5.0543660e-01 -1.3005082e-01]
  [ 3.9132228e-01 -9.9967159e-02]
  [ 3.0355835e-01 -7.2182484e-02]]

 [[ 1.4506341e-03 -3.7188528e-03]
  [-1.5539918e-03 -5.7408623e-03]
  [-1.1883080e-02 -9.8391389e-03]
  ...
  [ 2.0285213e+00 -4.0293849e-01]
  [ 1.6905962e+00 -3.3876693e-01]
  [ 1.4138532e+00 -2.7737367e-01]]

 [[ 2.8107704e-03 -4.0926212e-03]
  [ 3.9223276e-04 -6.4454712e-03]
  [-1.0385612e-02 -1.0507180e-02]
  ...
  [ 5.2403626e+00 -7.1162570e-01]
  [ 4.6022887e+00 -6.3034242e-01]
  [ 4.0175195e+00 -5.4961401e-01]]

 ...

 [[-3.3329110e-05  3.1431895e-04]
  [-3.4255107e-04  6.9793832e-04]
  [-5.3320901e-04  1.0452033e-03]
  ...
  [-4.4148430e-04 -8.5370522e-04]
  [-1.9479006e-04 -6.1119330e-04]
  [ 1.7827002e-06 -3.5627297e-04]]

 [[-7.9874357e-05  3.1680486e-04]
  [-3.2493132e-04  6.4680725e-04]
  [-4.8042162e-04  9.3844935e-04]
  ...
  [-3.9618849e-04 -4.4128596e-04]
  [-

In [3]:
import cv2
import numpy as np
import time
import imutils

def draw_flow(img, p0, p1, mask=None, color=(0, 255, 0), line_thickness=2):
    for i, (new, old) in enumerate(zip(p1, p0)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color, line_thickness)
        img = cv2.circle(img, (int(a), int(b)), 5, color, -1)
    return cv2.add(img, mask)

video_path = '.\\videos\\video2.mp4'
cap = cv2.VideoCapture(video_path)

suc, prev = cap.read()
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
valor = 600
prevgray = imutils.resize(prevgray, width=min(valor, prevgray.shape[1]))


feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
p0 = cv2.goodFeaturesToTrack(prevgray, mask=None, **feature_params)

mask = np.zeros_like(prev)

while True:
    suc, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = imutils.resize(gray, width=min(valor, gray.shape[1]))

    start = time.time()
    p1, st, err = cv2.calcOpticalFlowPyrLK(prevgray, gray, p0, None, winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    end = time.time()

    if end != start:
        fps = 1 / (end - start)
        print(f"{fps:.2f} FPS")
    else:
        print("Unable to calculate FPS (division by zero)")

    img = draw_flow(img, p0, p1, mask=mask)

    cv2.imshow('flow', img)
    p0 = p1.reshape(-1, 1, 2)

    key = cv2.waitKey(5)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


249.45 FPS
Unable to calculate FPS (division by zero)
Unable to calculate FPS (division by zero)
1000.79 FPS
994.62 FPS
903.17 FPS
999.60 FPS
992.26 FPS
999.36 FPS
Unable to calculate FPS (division by zero)
1948.12 FPS
995.80 FPS
Unable to calculate FPS (division by zero)
1000.07 FPS
991.56 FPS
Unable to calculate FPS (division by zero)
Unable to calculate FPS (division by zero)
Unable to calculate FPS (division by zero)
Unable to calculate FPS (division by zero)
Unable to calculate FPS (division by zero)
Unable to calculate FPS (division by zero)
Unable to calculate FPS (division by zero)
994.15 FPS
Unable to calculate FPS (division by zero)
Unable to calculate FPS (division by zero)
999.12 FPS
1000.07 FPS
Unable to calculate FPS (division by zero)
997.93 FPS
Unable to calculate FPS (division by zero)
Unable to calculate FPS (division by zero)
Unable to calculate FPS (division by zero)
998.64 FPS
958.70 FPS
998.88 FPS
998.88 FPS
Unable to calculate FPS (division by zero)
Unable to cal

In [12]:
import numpy as np
import cv2
import time
import imutils

# ... (definiciones de funciones omitidas)

video_path = '.\\videos\\video2.mov'
cap = cv2.VideoCapture(video_path)

suc, prev = cap.read()
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
valor = 800
prevgray = imutils.resize(prevgray, width=min(valor, prevgray.shape[1]))

while True:
    suc, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = imutils.resize(gray, width=min(valor, gray.shape[1]))

    # start time to calculate FPS
    start = time.time()

    flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    prevgray = gray

    # End time
    end = time.time()

    # calculate the FPS for current frame detection
    fps = 1 / (end - start)
    #print(f"{fps:.2f} FPS")

    # Visualizar el flujo óptico utilizando la función draw_hsv
    cv2.imshow('flow HSV', draw_hsv(flow))

    key = cv2.waitKey(5)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [13]:
import numpy as np
import cv2
import time
import imutils

# ... (definiciones de funciones omitidas)

video_path = '.\\videos\\video2.mov'
cap = cv2.VideoCapture(video_path)

suc, prev = cap.read()
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
valor = 800
prevgray = imutils.resize(prevgray, width=min(valor, prevgray.shape[1]))

# Variable de conteo para realizar el cálculo cada 10 frames
count = 0

In [14]:
def segmentar_movimiento(flow, umbral):
    magnitud = np.sqrt(flow[:,:,0]**2 + flow[:,:,1]**2)
    _, movimiento_binario = cv2.threshold(magnitud, umbral, 255, cv2.THRESH_BINARY)
    
    # Aplicar operaciones morfológicas (opcional)
    kernel = np.ones((5, 5), np.uint8)
    movimiento_binario = cv2.morphologyEx(movimiento_binario, cv2.MORPH_CLOSE, kernel)

    return movimiento_binario

while True:
    suc, img = cap.read()
    if not suc:
        break

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = imutils.resize(gray, width=min(valor, gray.shape[1]))

    # Incrementar el contador
    count += 1

    if count % 1 == 0:  # Realizar el cálculo cada 10 frames
        # start time to calculate FPS
        start = time.time()

        flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        prevgray = gray

        # Realizar la segmentación de movimiento
        umbral = 10  # Puedes ajustar este umbral según tus necesidades
        movimiento_binario = segmentar_movimiento(flow, umbral)

        # End time
        end = time.time()

        # calculate the FPS for current frame detection
        fps = 1 / (end - start)
        print(f"{fps:.2f} FPS")

        # Visualizar el flujo óptico y la segmentación de movimiento
        cv2.imshow('flow HSV', draw_hsv(flow))
        cv2.imshow('Segmentacion de movimiento', movimiento_binario)

    key = cv2.waitKey(5)
    if key == ord('q'):
        break



15.42 FPS
15.70 FPS
15.63 FPS
15.76 FPS
16.67 FPS
15.68 FPS
16.13 FPS
17.00 FPS
16.32 FPS
15.93 FPS
16.31 FPS
16.24 FPS
17.07 FPS
16.21 FPS
15.45 FPS
17.10 FPS
17.19 FPS
16.49 FPS
16.68 FPS
17.42 FPS
16.53 FPS
17.60 FPS
17.05 FPS
16.24 FPS
16.64 FPS
15.81 FPS
15.91 FPS
16.68 FPS
16.74 FPS
17.43 FPS
16.86 FPS
17.07 FPS
17.19 FPS
17.01 FPS
16.40 FPS
15.86 FPS
16.40 FPS
17.24 FPS
16.03 FPS
16.78 FPS
17.06 FPS
16.22 FPS
17.04 FPS
16.64 FPS
17.36 FPS
16.89 FPS
16.69 FPS
17.21 FPS
16.55 FPS
16.40 FPS
16.63 FPS
16.76 FPS
16.14 FPS
16.92 FPS
16.54 FPS
17.41 FPS
16.48 FPS
17.04 FPS
17.49 FPS
16.97 FPS
16.36 FPS
17.47 FPS
16.78 FPS
16.54 FPS
16.73 FPS
16.91 FPS
16.62 FPS
16.99 FPS
16.56 FPS
17.69 FPS
16.88 FPS
16.59 FPS
16.89 FPS
17.19 FPS
17.01 FPS
15.99 FPS
16.02 FPS
15.98 FPS
16.15 FPS
16.65 FPS
16.07 FPS
16.78 FPS
16.89 FPS
17.29 FPS
16.74 FPS
16.55 FPS
16.15 FPS
16.71 FPS
16.75 FPS
16.70 FPS
16.50 FPS
15.94 FPS
16.79 FPS
16.16 FPS
16.78 FPS
15.53 FPS
16.49 FPS
16.90 FPS
16.72 FPS
16.47 FPS


: 